In [1]:
import sys
sys.path.append('../../')

from os import listdir

from hdf5storage import loadmat, savemat 
import numpy as np 
from matplotlib import pyplot as plt

from scipy import signal 
from hnlpy.timeop import makefiltersos
from hnlpy.timeop import narrowfilter 
from scipy.signal import sosfiltfilt
from scipy.signal import hilbert
from scipy.signal import savgol_filter
from scipy.fftpack import fft
from scipy import stats # for zscoring

from sklearn.covariance import GraphicalLassoCV

from numpy.polynomial.polynomial import polyfit, polyval

import matplotlib.pyplot as plt

/home/zhibinz2/anaconda3/envs/eeg/lib/python3.9/site-packages/sklearn/utils/multiclass.py:13: DeprecationWarning: Please use `spmatrix` from the `scipy.sparse` namespace, the `scipy.sparse.base` namespace is deprecated.
  from scipy.sparse.base import spmatrix
/home/zhibinz2/anaconda3/envs/eeg/lib/python3.9/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/home/zhibinz2/anaconda3/envs/eeg/lib/python3.9/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, 

In [2]:
# outdict=loadmat('/home/zhibinz2/Documents/GitHub/Cleaned_data/hilbert_datacov/hilbert_dataCov_all.mat')

In [3]:
# hilbert_dataCov_all=outdict['hilbert_dataCov_all']

In [4]:
def avref(datax):
	ref = np.mean(datax,axis = 1)
	refmat = np.tile(ref,(np.shape(datax)[1],1))
	datax = datax - np.transpose(refmat)
	return datax

In [5]:
# build the narrow band filters
srnew = 200 
downsample = 10 
# passbands = [[1,3],[3.5,6.5],[7,10],[10.5,13.5],[14 ,20],[21,29],[30,49.5]]
# stopbands = [[0.75,3.25],[3.25,6.75],[6.75,10.25],[10.25,13.75],[13.75,20.5],[20.5,29.5],[29.5,50]]
passbands = [[3.5,6.5],[7,10],[10.5,13.5],[14 ,20],[21,29]]
stopbands = [[3.25,6.75],[6.75,10.25],[10.25,13.75],[13.75,20.5],[20.5,29.5]]
bandlabels = ['Delta','Theta','Alpha','Mu','Beta1','Beta2','Gamma']
esos = dict()
for j in range(len(passbands)):
    esos[j],w,h = narrowfilter(passbands[j],stopbands[j],samplingrate = srnew)
    #plt.plot(w[1:500],20*np.log10(np.abs(h[1:500])))


In [6]:
all_lambdas = np.flipud([0.6, 0.5, 0.4, 0.3, 0.2])

In [7]:
all_lambdas = np.flipud([0.6, 0.5, 0.4, 0.3, 0.2, 0.175, 0.15, 0.125, 0.1, 0.075, 0.05, 0.025, 0.01])

In [8]:
ncv = 5
pc_lasso = GraphicalLassoCV(cv=ncv, n_jobs=-10)

In [9]:
# organize file in time sequence
filedates=[20220713,20220721,20220804,20220808,20220810,20220811,20220815,20220816,20221003,2022100401,2022100402,20221005]
numSes=len(filedates)

In [10]:
ses=11
subj=1
tr=11
freq=4

In [11]:
filename='subj'+str(subj)+'_tr_'+str(tr)+'.mat'

In [12]:
outdict=loadmat('/ssd/zhibin/Cleaned_sourcedata/cortical_source_data/'+str(filedates[ses])+'/'+filename)

TypeError: Version.__init__() missing 1 required positional argument: 'version'

In [ ]:
agr_source_data=outdict['agr_source_data']

In [ ]:
# make a low pass filter @ 50 Hz 
sr=2000
sos_low, w,h = makefiltersos(sr,50,55)

In [ ]:
trialdata = avref(agr_source_data) 
trialdata = sosfiltfilt(sos_low,trialdata,axis=0)
nsamp=np.shape(trialdata)[0]
trialdatanew = trialdata[range(0,nsamp,downsample),:]

In [ ]:
freq=4
filtdata = sosfiltfilt(esos[freq],trialdatanew,axis = 0,padtype ='odd')


In [ ]:
hilbertdata = hilbert(filtdata,axis = 0)


In [ ]:
# Combine real and imaginary parts
sourceDataReal = np.concatenate((np.real(hilbertdata), np.imag(hilbertdata)), axis=1)

In [ ]:
# Normalize data
sourceDataReal = sourceDataReal * (1 / np.mean(np.abs(sourceDataReal)))

In [48]:
pip install cupy cuml
# 13 m

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 35.8 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Using cached fastrlock-0.8.2-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_28_x86_64.whl (51 kB)

In [20]:
import cupy as cp


ModuleNotFoundError: No module named 'cupy'

In [21]:
from cuml import Lasso

ModuleNotFoundError: No module named 'cuml'

In [35]:
pc_lasso.fit(sourceDataReal)
precision = pc_lasso.precision_
# 480s for 448 sources
# 19m 12.5s = 1153s for 896 sources
# reduce nv to 2 take 7 m 42 s = 460 s
# for 896 source with -10 jobs, it will takes 170
# for 896 source with -10 jobs and alphas of 10 values, it crashed
# for 896 source with -10 jobs and alphas from 0.2-0.6

/home/zhibinz2/anaconda3/envs/mne/lib/python3.10/site-packages/sklearn/covariance/_graph_lasso.py:245: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 598.0331922912177, tolerance: 0.2060832545194389
  coefs, _, _, _ = cd_fast.enet_coordinate_descent_gram(
/home/zhibinz2/anaconda3/envs/mne/lib/python3.10/site-packages/sklearn/covariance/_graph_lasso.py:245: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: nan, tolerance: 0.10342596235060458
  coefs, _, _, _ = cd_fast.enet_coordinate_descent_gram(
/home/zhibinz2/anaconda3/envs/mne/lib/python3.10/site-packages/sklearn/covariance/_graph_lasso.py:245: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: nan, tolerance: 0.029621558387140385
  coefs, _, _, _ = cd_fast.enet_coordinate_descent_gram(
/home/zhibinz2/anaconda3/envs/mne/lib/python3.10/site-pa

In [ ]:
import cupy as cp
from cuml import Lasso

# Assuming sourceDataReal is already on GPU (you may need to transfer it if it's on CPU)
# If it's not on GPU, you can use cp.asarray(sourceDataReal) to transfer it to GPU.

# Create a CuML Lasso model
pc_lasso = Lasso(alpha=1.0)

# Fit the model on GPU
pc_lasso.fit(cp.asarray(sourceDataReal))

# Get the precision on GPU
precision = cp.asnumpy(pc_lasso.precision_)


In [34]:
y = np.diag(precision)


In [35]:
yy = np.outer(y,y)


In [36]:
yy = np.sqrt(yy)


In [39]:
prec_mat = precision/yy

In [54]:
# time estimate (200 h)
12*2*12*5*180/3600/24

3.0

In [ ]:

for ses in range(12):
    for subj in range(2):
        for trial in range(12):
            for freq in range(5):
                
